# Paquetes y Funciones

In [1]:
import nbimporter
from funciones_scrapeo import *

In [2]:
load_dotenv()

True

# 1. Carga datos:

In [3]:
# Cargar las variables desde el archivo json
with open('datos_scrapers.json', 'rb') as f:
    datos = json.load(f)

In [4]:
# User agent:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

# Opciones:
opts = Options()
opts.add_argument("--disable-extensions")
opts.add_argument(f"User-Agent={user_agent}")

# OPENAI API KEY:
load_dotenv()
openai_api_key= os.getenv('OPENAI_APIKEY')
usuario = os.getenv('USUARIO_CORREO')
password = os.getenv('USUARIO_PASSWORD')
    
######################################################################
# Ruta de guardado:
ruta_datos = os.getcwd().replace("Codigo\\Scrapers", "Datos\\datos_sin_procesar\\")

# 2. Elegimos los portales que queremos scrapear:

In [6]:
dic_funciones = {#'ticjob' : [scraper_ticjob, None], 
                 'infoempleo' : [scraper_infoempleo, None], 
                 #'talenthacker' : [scraper_talenthacker, None], 
                 #'tecnoempleo' : [scraper_tecnoempleo, None], 
                 #'infojobs' : [scraper_infojobs, None], 
                 #'indeed' : [scraper_indeed, busqueda_indeed], 
                 #'linkedin' : [scraper_linkedin, busqueda_linkedin]
                }

In [7]:
datos['portales_busqueda']['linkedin']['dic_datos']['usuario'] = usuario
datos['portales_busqueda']['linkedin']['dic_datos']['password'] = password
datos['portales_busqueda']['infojobs']['dic_datos']['api_key'] = openai_api_key

In [8]:
empleos = datos['empleos_a_buscar']

In [7]:
fecha_scrapeo = datetime.now().date()

## 2.1. Ejecutamos para obtener datos:

In [ ]:
for portal, datos_portal in datos['portales_busqueda'].items():
    busqueda_por_empleo, busqueda_por_url, url, nombres_columnas, dic_datos = datos_portal.values()

    try:
        scraper, busqueda = dic_funciones[portal]
    except:
        continue
    
    #Abrimos el navegador y maximizamos la ventana
    driver = webdriver.Chrome(options=opts)
    driver.maximize_window()
    driver.get(url)
    sleep(2)
    
    #Busqueda y scrapeo de portales en los que hay que buscar empleos
    if busqueda_por_empleo:
        for empleo in empleos:
            try:
                busqueda(driver = driver, empleo = empleo, dic_datos = dic_datos)
            
            except:
                print(f'{portal} - {empleo} no scrapeado')
                continue
            
            scraper(empleo = empleo, driver = driver, fecha_scrapeo = fecha_scrapeo, portal = portal, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos, dic_datos = dic_datos)
            
            sleep(60)
            
            driver.get(url)
    
    #Busqueda y scrapeo de portales en los que NO hay que buscar empleos
    else:
        scraper(driver = driver, fecha_scrapeo = fecha_scrapeo, portal = portal, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos, dic_datos = dic_datos)
    
    driver.quit()
    
    df = pd.read_csv(ruta_datos + f'datos_{portal}.csv')
    
    procesar_datos_nuevos(df, portal, ruta_datos)